In [ ]:
using ProgressMeter
using Random #, Distributions
using PyPlot

In [ ]:
mutable struct PTParams{T}
    TMax       :: T
    λ          :: T
    NTemps     :: Int
    Nexchanges :: Int
    PTParams{T}() where {T} = new()
end;

mutable struct MetropolisParams{T}
    NSteps       :: Int
    StepSize     :: T
    GaussianStep :: Bool
    MetropolisParams{T}() where {T} = new()
end;

In [ ]:
function parallel_tempering(PTParameters::PTParams, MetropolisParams::MetropolisParams, initial_guess)

    # Define energy function

    Ener(x)          = -5*exp(-(x+3)^2) -exp(-2*(x-5.0)^2)

    # -------------------------------------------------------
    # Local variables
    # -------------------------------------------------------
    
    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))

    Eveco           = Ener.(xo)
    TupEBest        = findmin(Eveco)
    EBest           = TupEBest[1]
    EBestPos        = xo[TupEBest[2]]

    println("Initial guess: ", xo')
    println("Initial energy: ", EBest[1])
    println("Initial position: ", EBestPos)
    println("Temperatures: ", temperatures')
    # debug energy and init guess
    is_plot = false
    if is_plot
        clf()
        plot(-10:10, Ener.(-10:10), label="Energy")
        plot(xo, Ener.(xo), "ro", label="Initial guess")
        plot(EBestPos, EBest[1], "go", label="Best guess")
        title("Initial guess")
        xlabel("x")
        ylabel("Energy")
        legend()
        savefig("./out/energy")
    end

    # -------------------------------------------------------
    # Parallel tempering loop
    # -------------------------------------------------------
    @showprogress for _ in 1:PTParameters.Nexchanges+1
        # Metropolis step
        for i in 1:NSteps
            xn = xo .+ StepSize*randn(NTemps) # generate new positions
            Evecn = Ener.(xn) # compute the energy of each new position of each chain (NTemp long vector)
            ΔE_vec = Evecn .- Eveco # compute the energy difference between the new and old positions
            for i in 1:NTemps # loop over the chains (loop over the temperatures)
                if ΔE_vec[i] < 0 # if the new position is better, accept it
                    xo[i] = xn[i]
                elseif rand() < exp(-ΔE_vec[i]/temperatures[i]) # if the new position is worse, metropolis probability
                    xo[i] = xn[i]
                end
            end
            Eveco = Ener.(xo) # compute the energy of each updated (accepted/rejected) position of each chain (NTemp long vector)
            prob_best_guess = findmin(Eveco) # from the new sampled energies, find the best guess
            if prob_best_guess[1] < EBest # compare the new best guess with the old one. If its better, update it
                # EBest: energy, position
                # EBestPos: position
                EBest    = prob_best_guess[1]
                EBestPos = xo[prob_best_guess[2]]
            end
        end

        # Exchange step
        # no check for the lowest state, as we don't explore space here
        for temp in 1:NTemps-1
            ΔE_exchange = (Ener(xo[temp]) - Ener(xo[temp+1])) * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
                # Temp dependency is put inside ΔE_exchange!!!!!!
            elseif rand() < exp(-ΔE_exchange)
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
            end
        end
    end

    return xo, EBest, EBestPos
end;

In [44]:
kind  = Float32

Parms       = PTParams{kind}()
Parms.TMax  = 100
Parms.λ     = 0.92f0
Parms.NTemps     = 100
Parms.Nexchanges = 100

Metrop      = MetropolisParams{kind}()
Metrop.NSteps       = 100
Metrop.StepSize     = 0.1f0;

In [45]:
initial_layout = Array(5.0*randn(Parms.NTemps))

_,E,x=parallel_tempering(Parms,Metrop,initial_layout)

Initial guess: [8.098797939857077 1.4493041603322592 4.912866833132084 -0.7770062836442347 7.41283787160601 9.451285933866151 -0.8735624809837241 -2.79866532470507 -5.381858517537897 -5.358764408837082 -0.22294469150516114 5.669668278862447 -2.9526340942018297 8.723935170933943 2.0766708511580867 3.1657119423104314 0.4467148472054626 -0.6052486806639867 -1.1618502294321553 -0.39703950989571246 -5.049950564640282 0.7540633963076632 6.150839044262348 4.968720305701378 -6.7132627270539205 -2.9724284322434085 5.0459307139439815 1.1462208308289636 3.6514776591761473 5.330643511755614 5.12740389535978 -1.0930230970959647 -6.891789622146152 1.6368089003524813 2.1871752304943817 -2.094838961887867 -2.812849052672624 -1.6068709703525186 -2.4577270897947794 -4.217940946662468 1.120615693669681 -1.4545956059830492 -4.530972338861368 3.160609000652034 6.583843251490994 2.1344113001756946 -0.3035027964078812 10.633463529664278 -5.032113343588312 8.574428142138036 0.2794771593479239 0.19163230421093

([1.9894506705584771, 10.729111336516903, 7.45770371469514, 24.398862208304905, 22.81726529752454, 1.9536705272980188, -4.78821053456464, -1.3795637773691325, -5.814294105888402, 13.19947370093353  …  -3.025406751225401, 13.094811604460306, -2.9257153889409064, -11.923863613427931, 4.881370353444191, -3.1525248276910007, -2.967372833280943, 9.511638995392499, 0.9243722066445055, 35.02909561016448], -4.999999999994392, -3.0000010591065713)

In [46]:
println("Final guess: ", x')
println("Final energy: ", E[1])

Final guess: -3.0000010591065713
Final energy: -4.999999999994392


In [ ]:
function parallel_tempering_v2(initial_guess, Ener, PTParameters::PTParams, MetropolisParams::MetropolisParams)

    # Define energy function

    # Ener(x)          = -5*exp(-(x+3)^2) -exp(-2*(x-5.0)^2)

    # -------------------------------------------------------
    # Local variables
    # -------------------------------------------------------
    
    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    TMin             = PTParameters.TMin
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))

    Eveco           = Ener.(xo)
    TupEBest        = findmin(Eveco)
    EBest           = TupEBest[1]
    EBestPos        = xo[TupEBest[2]]

    function metropolis_step(xo, Eveco)
        xn = xo .+ StepSize*randn(NTemps) # generate new positions
        Evecn = Ener.(xn) # compute the energy of each new position of each chain (NTemp long vector)
        ΔE_vec = Evecn .- Eveco # compute the energy difference between the new and old positions
        for i in 1:NTemps # loop over the chains (loop over the temperatures)
            if ΔE_vec[i] < 0 # if the new position is better, accept it
                xo[i] = xn[i]
            elseif rand() < exp(-ΔE_vec[i]/temperatures[i]) # if the new position is worse, metropolis probability
                xo[i] = xn[i]
            end
        end
        return xo
    end

    function update_best_energy(Eveco, EBest, EBestPos)
        new_best_guess = findmin(Eveco) # from the new sampled energies, find the best guess
        if new_best_guess[1] < EBestPos
            return new_best_guess[1], xo[new_best_guess[2]]
        else
            return EBest, EBestPos
        end
        
    end

    # -------------------------------------------------------
    # Parallel tempering loop
    # -------------------------------------------------------
    @showprogress for _ in 1:PTParameters.Nexchanges+1
        # Metropolis step
        for _ in 1:NSteps

            # Metropolis step-------------------------------------------
            xo = metropolis_step(xo, Eveco)
            # Metropolis step-------------------------------------------

            # Best Energy update.............................................
            Eveco = Ener.(xo) # compute the energy of each updated (accepted/rejected) position of each chain (NTemp long vector)
            EBest, EBestPos = update_best_energy(Eveco, EBest, EBestPos)
            # Best Energy update.............................................
        end

        # Exchange step
        # no check for the lowest state, as we don't explore space here
        for temp in 1:NTemps-1
            ΔE_exchange = (Ener(xo[temp]) - Ener(xo[temp+1])) * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
                # Temp dependency is put inside ΔE_exchange!!!!!!
            elseif rand() < exp(-ΔE_exchange)
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
            end
        end
    end

    return xo, EBest, EBestPos
end;

In [50]:
Ener(x)          = -5*exp(-(x+5)^2) -exp(-2*(x-5.0)^2)
_, E,x=parallel_tempering_v2(initial_layout, Ener, Parms, Metrop)
println("Final guess: ", x')
println("Final energy: ", E)

Initial guess: [12.952825160741519 2.0813231725594266 34.42046387181929 -2.0533890246771254 -4.992939717291036 2.407102120835536 -19.48538291945678 -5.54464206184096 -2.8150315122745413 -10.06504780836783 -2.63941046201513 12.32227238589645 -6.146327310163745 -1.5747763954033625 8.152386617842483 20.52580765023945 -5.468507845194244 20.3802016066969 -10.055138661736775 -8.661282321546606 17.77770820184163 7.588908934155058 9.493980111132101 -5.6552230958322545 -5.000222031619426 12.837174038712602 -24.683639984460132 3.6108421898779386 -3.0217476400790524 -4.38938755051099 -3.314211385375386 27.33917056715085 4.357863812632949 13.527799779850085 -3.39387771570071 -6.665845978757807 -5.836739560528758 -7.120136532128251 -3.0328702499550895 -0.8106476209338461 0.6501286886106559 10.746785723408971 15.958722969752102 -2.1552126695232596 15.31406523710693 -2.8116343988736214 37.452653090028264 -19.408611125593804 -18.99786427831462 9.722333696900238 12.173033288426366 -2.9575708717892955 -

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Final guess: -5.000222031619426
Final energy: -4.999999753509806


In [ ]:
xn = Array([1,2,3])
xo = Array([3,2,1])

temperatures = [1,1,1]

ΔE_vec = xn.^2 .- xo.^2

condition1 = ΔE_vec .< 0
condition2 = .!condition1 .& (rand(3) .< exp.(-ΔE_vec./temperatures))
condition = condition1 .| condition2

println(condition')
println(xn')
println(xo')

println([xn[i] for i in 1:3 if condition[i]])

xo .= xn.*condition .+ xo.*.!condition

Bool[1 1 0]
[1 2 3]
[3 2 1]
[1, 2]


3-element Vector{Int64}:
 1
 2
 1

In [ ]:


struct Exchanges
    Texchange::Vector
    ExchangeStep::Vector{Int}
end

"""
# Parallel Tempering MCMC (PTMCMC)

with PTParams we define the number of temperatures, the max and min of them and generate a linspace of temperatures. Also we define how many times do we want to exchange the temperatures.
with metropolisparams we can define the number of steps that we want to do in each time we sample from metropolis (aka the steps before an exchange)

Should I use dictionaries as inputs instead of structs? maybe dicts are more flexible...

"""

function parallel_tempering(exchanges, PTParameters::PTParams, MetropolisParams::MetropolisParams, f, initial_guess,args...)
    NTemps       = PTParameters.NTemps
    Nexchanges   = PTParameters.Nexchanges
    TMax         = PTParameters.TMax
    TMin         = PTParameters.TMin
    NSteps       = MetropolisParams.NSteps
    StepSize     = MetropolisParams.StepSize
    GaussianStep = MetropolisParams.GaussianStep
    # temperatures = range(TMin, TMax, length=NTemps)
    # TN=T0*λ^N
    # TODO: we can have an arbitrary way of choosing the temperatures, Let the user decide.
    
    λ               = PTParameters.λ
    temperatures    = zeros(NTemps)
    temperatures[1] = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]
    
        display(temperatures)
        error("aki")
    
    #temperatures = TMax * (TMin/TMax).^(range(0, NTemps-1, length=NTemps))
    # n = range(0, NTemps-1, length=NTemps)/(NTemps-1)
    # temperatures = TMax.^(1 .-n).*TMin.^n
    plot(temperatures)
    title("Tmin=$(minimum(temperatures)) Tmax=$TMax")
    # logscale in y axis
    yscale("log")
    savefig("out/temperatures.png")
    clf()
    x = initial_guess*ones(NTemps)
    # x = x .+ randn(NTemps)
    d = Normal(0,1)
    x = x .+ rand(d, NTemps)
    @info "Temperatures: $temperatures"
    @info "Initial guess: $x"
    total_chains = zeros(Nexchanges+1,NTemps, NSteps)
    total_chains[1,:,1] = x
    
    count = 0
    
    @showprogress for step in 1:PTParameters.Nexchanges+1
        # Sampling step
        Threads.@threads for i in 1:NTemps # parallel!!!!!!!

            # TODO: can i change step characteristics to explore the space better?
            # propose a state and metropolis accept reject. split into step choosing and metropolis accepting. Thought only to have energies, 
            # so everything is an exponential
            x[i], total_chains[step ,i, :] = metropolis_energy(NSteps, f, x[i], StepSize, GaussianStep, temperatures[i], args...)
        end
        # Exchange step
        for i in 1:NTemps-1 # starting from highest temperature, going to lowest
            # exchange for temperature i and i-1
            ΔE_exchange = (f(total_chains[step, i, end], args...) - f(total_chains[step, i+1, end], args...)) * (1/temperatures[i] - 1/temperatures[i+1])
            ΔE_exchange = -f(total_chains[step, i, end], args...)/temperatures[i+1] - f(total_chains[step, i+1, end], args...)/temperatures[i]+f(total_chains[step, i, end], args...)/temperatures[i] + f(total_chains[step, i+1, end], args...)/temperatures[i+1]
            if ΔE_exchange < 0
                x[i], x[i+1] = x[i+1], x[i]
                count += 1
            elseif rand() < exp(-ΔE_exchange)
                x[i], x[i+1] = x[i+1], x[i]
            end
        end
    end
    print(count)
    return total_chains[end,end,end], total_chains
end

function flatten_chains(chains, temp_idx)
    temp_chain = []
    for j in 1:size(chains,1)
        append!(temp_chain, chains[j,temp_idx,:])
    end
    return temp_chain[2:end]
end

gaussian_energy_landscape(x,args...) = -5*exp(-(x+2)^2) -exp(-2*(x-5.0)^2)
plot_landscapes = true
using PyPlot
clf()
if plot_landscapes
    x = range(-10, 10, length=1000)
    plot(x, gaussian_energy_landscape.(x))
    # plot a vertical line in the minimum
    axhline(y=minimum(gaussian_energy_landscape.(x)), color="red", linestyle="--")
    savefig("out/energy_landscape.png")
    clf()
end

#lets find the min with parallel tempering

initial_guess = -1.0
λ = 0.7
TMax = 100.0
TMin = TMax*λ
NTemps = 50
Nexchanges = 100
NSteps = 100
StepSize = 0.01
GaussianStep = true

PTParameters = PTParams(TMax, TMin, NTemps, Nexchanges)
MetropolisParameters = MetropolisParams(NSteps, StepSize, GaussianStep)

exchanges = Exchanges([], [])

quadratic_energy_landscape(x,args...) = x^2

final, chains = parallel_tempering(exchanges, PTParameters, MetropolisParameters, gaussian_energy_landscape, initial_guess)

chains1 = flatten_chains(chains, 1)


# for i in 1:NTemps
#     plot(chains[:,i,:], label="T = $(round(range(TMax, TMin, length=NTemps)[i], digits=2))")
# end

# plot(chains1, label="T = $(round(range(TMax, TMin, length=NTemps)[1], digits=2))")

# chains2 = flatten_chains(chains, 2)
# plot(chains2, label="T = $(round(range(TMax, TMin, length=NTemps)[2], digits=2))")

# chains3 = flatten_chains(chains, 3)
# plot(chains3, label="T = $(round(range(TMax, TMin, length=NTemps)[3], digits=2))")

# lowest_temp_chain = flatten_chains(chains, NTemps)
# sec_lowest_temp_chain = flatten_chains(chains, NTemps-1)
# plot(sec_lowest_temp_chain[size(sec_lowest_temp_chain,1)-2*NSteps:end], label="T = $(round(range(TMax, TMin, length=NTemps)[NTemps-1], digits=2))")

# plot(lowest_temp_chain[size(lowest_temp_chain,1)-2*NSteps:end], label="T = $(round(range(TMax, TMin, length=NTemps)[NTemps], digits=2))")

# for i in NTemps-2:NTemps
temperatures = TMax * (TMin/TMax).^(range(0, NTemps-1, length=NTemps))

temp_labels = round.(temperatures, digits=2)

for i in NTemps-1:NTemps
    plot(flatten_chains(chains, i), label="T = $(temp_labels[i])")
end

title("$final")
legend()
savefig("out/chain.png")
close(io)

# using PyCall
# @pyimport matplotlib.animation as anim
# using PyPlot


# fig, ax = PyPlot.subplots(nrows=1, ncols=1, figsize=(7, 2.5))
# # ax1, ax2 = axes

# low_T_chain = flatten_chains(chains, NTemps)

# println(size(chains))

# println(size(low_T_chain))

# println()


# function make_frame(i)
#     ax.clear()
#     ax.set_title("$(i+1)")
#     ax.plot(x, gaussian_energy_landscape.(x))
#     ax.plot(low_T_chain[i+1], gaussian_energy_landscape(low_T_chain[i+1]), "ro")
    
#     # ax1.clear()
#     # ax2.clear()
#     # ax1.imshow(A[:,:,i+1, 1])
#     # ax2.imshow(A[:,:,i+1, 2])
# end

# N_iter_per_temp = size(low_T_chain,1)

# frames = [N_iter_per_temp-200:N_iter_per_temp-1]

# myanim = anim.FuncAnimation(fig, make_frame, frames=frames, interval=20, blit=false)
# myanim[:save]("test2.gif", bitrate=-1)